https://github.com/uscensusbureau/acs-summary-file/wiki/Python-Table-Data-for-All-Tracts

In [6]:
import pandas as pd
from ftplib import FTP
import os, sys

In [28]:
def table_for_sumlevel(tblid, year, dataset, sumlevel):

    #create output directory. 
    outdir = 'output'
    try:
        os.mkdir(outdir)
    except FileExistsError as e:
        print(f"directory named '{outdir}' already exists. delete it and try again.")
        sys.exit(1)

    dir =f"/programs-surveys/acs/summary_file/{year}/prototype/{dataset}YRData/"

    #go to ftp site
    ftp = FTP("ftp2.census.gov")
    ftp.login("","")
    ftp.cwd(dir)

    #get .dat file based on tblid or all tables
    files = [x for x in ftp.nlst() if f"{tblid}.dat" in x or (tblid=="*" and ".dat" in x)]

    for file in files:
        #read data file and query for summary level (http faster than ftp)
        df = pd.read_csv(f"https://www2.census.gov{dir}{file}", sep="|")
        df = df[ df['GEO_ID'].str.startswith(sumlevel) ]

        #output
        if not df.empty:
            df.to_csv(f"{outdir}/{file}", sep="|", index=False)
            print(f"{outdir}/{file} output.")
            return hello

[Summary level definitions](https://mcdc.missouri.edu/geography/sumlevs/#:~:text=the%20U.S.-,Summary%20Levels,-All%20MCDC%20census)

In [26]:
#get all tables for all tracts

table_for_sumlevel(tblid = 'b16001', year=2020, dataset=5, sumlevel='50')

directory named 'output' already exists. delete it and try again.


AssertionError: 

I need to re-do this workflow and instead of saving a .dat to an output directory and re-loading it, I can directly load it as a df

In [34]:
df = pd.read_csv('output/acsdt5y2020-b16001.dat', delimiter='|')
df

,GEO_ID,B16001_E001,B16001_M001,B16001_E002,B16001_M002,B16001_E003,B16001_M003,B16001_E004,B16001_M004,B16001_E005,...,B16001_E124,B16001_M124,B16001_E125,B16001_M125,B16001_E126,B16001_M126,B16001_E127,B16001_M127,B16001_E128,B16001_M128
0,5001600US0101,670943,670,642742,2068,14379,1225,9218,970,5161,...,87,72,7,12,87,79,87,79,0,29
1,5001600US0102,638466,2536,608731,2666,17725,901,11123,821,6602,...,115,69,0,29,407,192,227,116,180,141
2,5001600US0103,671514,1904,640437,2606,15829,1063,10282,926,5547,...,117,89,29,36,93,72,93,72,0,29
3,5001600US0104,645746,2146,605502,2445,33753,1062,17412,1034,16341,...,12,15,0,29,185,130,111,78,74,100
4,5001600US0105,686821,286,644214,1833,26301,1118,14719,1116,11582,...,199,94,0,29,591,352,250,172,341,210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,5001600US5506,676547,809,633825,1341,19670,866,12794,665,6876,...,229,126,0,22,283,172,272,172,11,18
436,5001600US5507,673002,778,641681,1224,9975,544,5913,431,4062,...,947,147,159,54,254,100,204,72,50,51
437,5001600US5508,691273,423,644420,1649,26974,1303,15597,918,11377,...,649,162,61,38,101,53,92,51,9,15
438,5001600US5600,545912,410,507936,1608,25717,1344,17581,1189,8136,...,1894,262,81,39,296,161,278,142,18,27


Table header labels [here](https://www2.census.gov/programs-surveys/acs/summary_file/2020/prototype/)

In [41]:
df2 = pd.read_csv('ACS2020_Table_Shells.csv', encoding_errors="replace")
labels = df2[df2['Table ID'] == 'B16001']
labels

,Table ID,Line,Unique ID,Stub,Data Release
12470,B16001,,NaN,LANGUAGE SPOKEN AT HOME BY ABILITY TO SPEAK EN...,"1,5"
12471,B16001,,NaN,Universe: Population 5 years and over,NaN
12472,B16001,1,B16001_001,Total:,NaN
12473,B16001,2,B16001_002,Speak only English,NaN
12474,B16001,3,B16001_003,Spanish:,NaN
...,...,...,...,...,...
12595,B16001,124,B16001_124,"Speak English ""very well""",NaN
12596,B16001,125,B16001_125,"Speak English less than ""very well""",NaN
12597,B16001,126,B16001_126,Other and unspecified languages:,NaN
12598,B16001,127,B16001_127,"Speak English ""very well""",NaN
